https://www.kaggle.com/iafoss/panda-16x128x128-tiles

In [1]:
import os
import cv2
import skimage.io
from tqdm.notebook import tqdm
import zipfile
import numpy as np

import sys
sys.path.append("../utils")
from minimal_image import get_compact

import json

In [2]:
TRAIN = '../input/prostate-cancer-grade-assessment/train_images/'
MASKS = '../input/prostate-cancer-grade-assessment/train_label_masks/'

OUT_TRAIN = '../input/panda-16x128x128-tiles-data/train_1024'
sz = 1024
N = 16

In [3]:
def tile(img):
    result = []
    shape = img.shape
    pad0,pad1 = (sz - shape[0]%sz)%sz, (sz - shape[1]%sz)%sz
    
    img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],
                constant_values=255)
    
    h, w = img.shape[:2]
    
    img = img.reshape(img.shape[0]//sz,sz,img.shape[1]//sz,sz,3)
    img = img.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
    
    if len(img) < N:
        img = np.pad(img,[[0,N-len(img)],[0,0],[0,0],[0,0]],constant_values=255)

    idxs = np.argsort(img.reshape(img.shape[0],-1).sum(-1))[:N]
    
    img = img[idxs]
    for i in range(len(img)):
        result.append({'img':img[i], 'idx':i})
    
    return result

In [4]:
with open("../input/compact_representation.json", "r") as file:
    compact_representation = json.load(file)

In [5]:
names = [name[:-10] for name in os.listdir(MASKS)]

In [8]:
name

'00928370e2dfeb8a507667ef1d4efcbb'

In [7]:
os.makedirs(OUT_TRAIN, exist_ok=True)

x_tot,x2_tot = [],[]
for name in tqdm(names):
    key = name + '.tiff'
    img = skimage.io.MultiImage(os.path.join(TRAIN, name + '.tiff'))[0]

    img = get_compact(img, compact_representation[key])

    tiles = tile(img)
    for t in tiles:
        img, idx = t['img'], t['idx']
        x_tot.append((img/255.0).reshape(-1,3).mean(0))
        x2_tot.append(((img/255.0)**2).reshape(-1,3).mean(0)) 
        #if read with PIL RGB turns into BGR
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        cv2.imwrite(os.path.join(OUT_TRAIN, f'{name}_{idx}.png'), img)

error: OpenCV(4.2.0) /io/opencv/modules/imgproc/src/imgwarp.cpp:1724: error: (-215:Assertion failed) dst.cols < SHRT_MAX && dst.rows < SHRT_MAX && src.cols < SHRT_MAX && src.rows < SHRT_MAX in function 'remap'


In [ ]:
#image stats
img_avr =  np.array(x_tot).mean(0)
img_std =  np.sqrt(np.array(x2_tot).mean(0) - img_avr**2)
print('mean:',img_avr, ', std:', np.sqrt(img_std))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
for name in tqdm(names[:10]):
    key = name + '.tiff'
    img = skimage.io.MultiImage(os.path.join(TRAIN, name + '.tiff'))[0]

    img = get_compact(img, compact_representation[key])

    tiles = tile(img)
    
    n = int(np.sqrt(N))
    res_img = np.empty([sz * n, sz * n, 3], dtype=np.uint8)
    for i in range(N):
        t = tiles[i]
        idx = t["idx"]
        x = idx % n
        y = idx // n
        res_img[y*sz:(y + 1)*sz, x*sz:(x+1)*sz] = t["img"]
    plt.figure()
    plt.imshow(res_img)

In [ ]:
mean_h_o = 0
mean_w_o = 0
mean_h = 0
mean_w = 0
for key, data in compact_representation.items():
    oh, ow = data['original_size']
    rh, rw = 0, 0
    for _, (w, h), _ in data['rectangles']:
        rh += h
        rw = max(rw, w)
    mean_h_o += oh
    mean_w_o += ow
    
    mean_h += h
    mean_w += w
mean_h /= len(compact_representation)
mean_w /= len(compact_representation)
mean_h_o /= len(compact_representation)
mean_w_o /= len(compact_representation)
factor = 16
[mean_h * factor, mean_w * factor], [mean_h_o * factor, mean_w_o * factor]

In [ ]:
sz * sz * N / (mean_h * mean_w * 16)

In [ ]:
# a = skimage.io.MultiImage(os.path.join(TRAIN, name + '.tiff'))[0].shape
# b = skimage.io.MultiImage(os.path.join(TRAIN, name + '.tiff'))[1].shape
# c = skimage.io.MultiImage(os.path.join(TRAIN, name + '.tiff'))[2].shape
# a, b, c

In [ ]:
22528 / 5632, 22528 / 1408